In [ ]:
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("whitegrid")

nice_colors = {
    "RES": "limegreen",
    "Storage": "darkgrey",
    "Electrolysis": "lightcoral",
    "Transport": "blue",
    "Electrification": "coral",
}

# Select data to plot
df = pd.read_csv(snakemake.input["results"], delimiter=";")

df = df.loc[
    (df["exporter"] == snakemake.wildcards["exporter"])
    & (df["esc"] == snakemake.wildcards["esc"])
]

df["flexibility"] = df["scenario"].str.replace("_.*", "", regex=True)
df["csp"] = df["scenario"].str.contains("with_csp")

# Add sorted category column to df indicating type of flexibility
# then sort by that category to ensure correct order in x-axis of plot
# Note: Reproduces the order given in "flexibilities".

flexibilities = {
    "unbuffered": "No flexibility",
    "daily": "Daily",
    "weekly": "Weekly",
    "biweekly": "Biweekly",
    "monthly": "Month",
    "quaterly": "Quaterly",
    "annually": "Yearly",
}

df["flexibility"] = df.flexibility.replace(flexibilities)

df["flexibility"] = pd.Categorical(
    df.flexibility, categories=flexibilities.values(), ordered=True
)

df = df.sort_values("flexibility")

demand_in_MWh = df.query("subcategory == 'Total demand'")["value"].unique().item()

df = df.query("category == 'cost'")[["csp", "flexibility", "subcategory", "value"]]

# Get rid of numbers of technologies
df["subcategory"] = df.subcategory.str.replace("\s*\d+\s*", "", regex=True)

aggregate_technologies = {
    "Buffer: electricity demand": "Storage",
    "CCGT (exp)": "Electrification",
    "CCGT (imp)": "Electrification",
    "H(g) fill compressor station (exp)": "Transport",
    "H(g) pipeline": "Transport",
    "H(g) pipeline decompressor (imp)": "Transport",
    "H(g) submarine pipeline": "Transport",
    "HVDC inverter pair (exp)": "Transport",
    "HVDC inverter pair (imp)": "Transport",
    "HVDC overhead": "Transport",
    "HVDC submarine": "Transport",
    "Hstorage storing (exp)": "Storage",
    "Hstorage storing (imp)": "Storage",
    "Hstorage unstoring (exp)": "Storage",
    "Hstorage unstoring (imp)": "Storage",
    "battery inverter (charging, exp)": "Storage",
    "battery inverter (charging, imp)": "Storage",
    "battery inverter (discharging, exp)": "Storage",
    "battery inverter (discharging, imp)": "Storage",
    "battery storage": "Storage",
    "clean water tank storage": "Storage",
    "csp-tower": "RES",
    "csp-tower TES": "Storage",
    "csp-tower power block": "RES",
    "electrolysis (exp)": "Electrolysis",
    "electrolysis (imp)": "Electrolysis",
    "hydrogen storage tank type": "Storage",
    "offwind": "RES",
    "onwind": "RES",
    "seawater desalination (exp)": "Electrolysis",
    "solar-utility": "RES",
}

# Replace category names with names/techs to aggregate on, then aggregate
df["technology"] = df["subcategory"].replace(aggregate_technologies)
df = (
    df.groupby(["csp", "flexibility", "technology"])["value"]
    .sum()
    .to_frame()
    .reset_index()
)

# Rearrange for plotting
df = df.pivot(index=["csp", "flexibility"], columns="technology", values="value")

# Which technologies (columns) from dataframe to plot and in which order (bottom to top) in stacked bar plot
technologies_to_plot = [
    "RES",
    "Storage",
    "Electrolysis",
    "Transport",
    "Electrification",
]

# Apply Column/Plot order of technologies
df = df[[t for t in technologies_to_plot if t in df.columns]]

# Up till now cost in total for full demand, present results in share of cost per MWh: Calculate here
df = df / demand_in_MWh

# Plotting
fig, ax = plt.subplots(figsize=(7, 3))

df.xs(False, level="csp").plot.bar(
    stacked=True,
    position=1,
    width=0.3,
    color=[nice_colors[t] for t in df.columns],
    ax=ax,
)
df.xs(True, level="csp").plot.bar(
    stacked=True,
    position=0,
    width=0.3,
    color=[nice_colors[t] for t in df.columns],
    hatch="///",
    ax=ax,
)

# Custom legend: Two columns with heading

# For the two headings, add two empty handles
empty_handles = [ax.plot([], marker="", ls="")[0]] * 2

no_labels = int(len(ax.get_legend_handles_labels()[0]) / 2)

# Construct order such that legend order of colors matches color bar stacking
new_handles = (
    empty_handles[:1]
    + ax.get_legend_handles_labels()[0][:no_labels][::-1]
    + empty_handles[1:]
    + ax.get_legend_handles_labels()[0][no_labels:][::-1]
)

# Pad legend labels to this length such that legend is always same width for all
# ESCs (technologies), even if the technology is not in the ESC
pad_to_length = np.max([len(t) for t in technologies_to_plot])
# Reorder labels accordingly and add color headings
new_labels = (
    ["w/o"]
    + [""] * no_labels
    + ["w/     CSP"]  # spaces to align with text of other labels
    + [
        l.ljust(pad_to_length)
        for l in ax.get_legend_handles_labels()[1][no_labels:][::-1]
    ]
)

# Add custom legend
legend = ax.legend(
    handles=new_handles,
    labels=new_labels,
    title=None,
    fontsize="small",
    # markerfirst=False,
    ncol=2,
    columnspacing=0.0,
    bbox_to_anchor=(1.0, 0),
    loc="lower left",
)
# Remove spacing such that column headings are above color handles
for vpack in legend._legend_handle_box.get_children():
    for hpack in vpack.get_children()[:1]:
        hpack.get_children()[0].set_width(-5)

ax.set_xlabel("Flexibility to shift load")
ax.set_ylabel("Cost share [EUR/MWh]")

ax.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize="small")
ax.set_yticks([0, 50, 100, 150, 200])
ax.tick_params(axis="both", which="major", labelsize="small")

ax.set_ylim(0, 205)  # Magic number 200: Highest value from TN ESC H2 pipeline
ax.set_xlim(
    -0.5,
)

fig.tight_layout()

# Saving figure
fig.savefig(snakemake.output["pdf"], dpi=300, bbox_inches="tight")
fig.savefig(snakemake.output["png"], dpi=300, bbox_inches="tight")